<a href="https://colab.research.google.com/github/AnujM11/Tensorflow/blob/main/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip uninstall tensorflow

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.12.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
Y
  Successfully uninstalled tensorflow-2.12.0


In [76]:
pip install tensorflow==2.10


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 53.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0

In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.10.0


In [3]:
import urllib
import zipfile
import pandas as pd
import numpy as np

In [4]:
def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()

In [5]:
download_and_extract_data()

In [6]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

In [44]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    print(ds)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    ds = ds.batch(batch_size).prefetch(1)
    print(ds)
    return ds




In [7]:
df = pd.read_csv('household_power_consumption.csv', sep=',',infer_datetime_format=True, index_col='datetime', header=0)

In [8]:
N_FEATURES = len(df.columns) # DO NOT CHANGE THIS

In [9]:
N_FEATURES

7

In [10]:
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

In [11]:
SPLIT_TIME = int(len(data) * 0.5) # DO NOT CHANGE THIS
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

# DO NOT CHANGE THIS CODE
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [12]:
BATCH_SIZE = 32
N_PAST = 24
N_FUTURE = 24
SHIFT = 1

In [15]:
ds = tf.data.Dataset.from_tensor_slices(x_train)
print(ds)
ds = ds.window(size=N_PAST + N_FUTURE, shift=SHIFT, drop_remainder=True)
ds = ds.flat_map(lambda w: w.batch(N_PAST + N_FUTURE))
ds = ds.map(lambda w: (w[:N_PAST], w[N_PAST:]))
ds = ds.batch(BATCH_SIZE).prefetch(1)

<TensorSliceDataset element_spec=TensorSpec(shape=(7,), dtype=tf.float64, name=None)>


In [81]:
l0= tf.keras.layers.Dense(1,input_shape=[7])

In [82]:
model = tf.keras.models.Sequential([l0])

In [91]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True,input_shape=(32,24))),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(1)
])

In [92]:
model.compile(loss='mse', optimizer='adam')

In [93]:
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

InvalidArgumentError: ignored

In [85]:
print("Layer weight{}".format(l0.get_weights()))

Layer weight[array([[ 0.16945612],
       [-0.1438804 ],
       [ 0.37597582],
       [ 0.03515528],
       [-0.326117  ],
       [ 0.4149406 ],
       [ 0.12411109]], dtype=float32), array([0.06097257], dtype=float32)]
